In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install lungs-segmentation
!pip install opencv-python==3.4.5.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.0 MB 77.9 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 789 kB 71.4 MB/s 
     |████████████████████████████████| 28.2 MB 73.3 MB/s 
     |████████████████████████████████| 12.8 MB 22.0 MB/s 
     |████████████████████████████████| 13.1 MB 57.4 MB/s 
     |████████████████████████████████| 72 kB 877 kB/s 
     |████████████████████████████████| 47.8 MB 46 kB/s 
     |████████████████████████████████| 948 kB 58.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: opencv-python
    Found existing in

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 101.9 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.2.0.34
    Uninstalling opencv-python-4.2.0.34:
      Successfully uninstalled opencv-python-4.2.0.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lungs-segmentation 1.1.2 requires opencv-python==4.2.0.34, but you have opencv-python 3.4.5.20 which is incompatible.


In [6]:
import numpy as np
import torch
import cv2
from lungs_segmentation.pre_trained_models import create_model
import lungs_segmentation.inference as inference
import glob
import os
import matplotlib.pyplot as plt

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [20]:
%cd "/content/drive/MyDrive/CXR/Resized/ALL/1"

/content/drive/MyDrive/CXR/Resized/ALL/1


In [21]:
# load images
image_list = glob.glob('./*.JPG') + glob.glob('./*.jpg') + glob.glob('./*.jpeg')
# image_list = glob.glob('./*.jpg')
# image_list = glob.glob('./*.jpeg')
image_names = [i.split('/')[-1] for i in image_list]

In [22]:
len(image_names)

734

In [10]:
model = create_model("resnet34")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Downloading: "https://github.com/alimbekovKZ/lungs_segmentation/releases/download/1.0.0/resnet34.pth" to /root/.cache/torch/hub/checkpoints/resnet34.pth


  0%|          | 0.00/98.6M [00:00<?, ?B/s]

In [23]:
# os.mkdir('./masked_images')

for img_name in image_names:
  # inference
  image, mask = inference.inference(model,img_name, 0.2)
  # mask images
  image_masked = np.zeros_like(image)
  image_masked[np.where(np.logical_or(mask[0], mask[1]))] = image[np.where(np.logical_or(mask[0], mask[1]))]
  image_masked = image_masked.astype(np.float64) / image_masked.max() # normalize the data to 0 - 1
  image_masked = 255 * image_masked # Now scale by 255
  image_masked = image_masked.astype(np.uint8)
  cv2.imwrite(os.path.join('/content/drive/MyDrive/CXR/Resized/HypothesisTesting/AllSegmented/1', img_name), image_masked)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [25]:
# import os
import csv
counter = 0
for img in os.listdir("/content/drive/MyDrive/CXR/Resized/HypothesisTesting/AllSegmented/1"):
  counter = counter + 1
print(counter)

734
